<a href="https://colab.research.google.com/github/carmeloarci/carmeloarci/blob/main/AFOSC_POL_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# =============================================================================
# AFOSC_POL_pipe.ipynb
# =============================================================================
# Scopo:
#
# Il codice realizza le procedure di riduzione e analisi dei dati polarimetrici acquisiti con AFOSC
# (Asiago Faint Object Spectrograph and Camera) installato al telescopio Copernico (1.82 m, Cima Ekar).
# In particolare, il notebook permette:
#   - la calibrazione fotometrica e polarimetrica delle immagini,
#   - l’estrazione dei parametri di Stokes (I, Q, U),
#   - la ricostruzione del grado e dell’angolo di polarizzazione lineare,
#   - la visualizzazione diagnostica dei risultati.
#
#
# Autore: Dr Matteo Simioni, Dr Luca Cortese, Dr. Carmelo Arcidiacono
# INAF – Osservatorio Astronomico di Padova
# Data: Ottobre 2025
# =============================================================================# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Install missing package
!pip install photutils astropy matplotlib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.stats import SigmaClip
from photutils.background import MedianBackground
from photutils.segmentation import detect_sources, SourceCatalog
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from scipy.spatial import cKDTree
from datetime import datetime
import matplotlib.patches as patches
import ipywidgets as widgets
from IPython.display import display, clear_output
from photutils.centroids import centroid_com
from astropy.stats import SigmaClip
import numpy as np
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
import gc
from astropy.stats import sigma_clip

In [ ]:


# --- FUNCTIONS ---
# ---------------------------------------------------------------------------
# detect_sources_in_image
# ---------------------------------------------------------------------------
# Detects astronomical sources within predefined Regions of Interest (ROIs)
# in the AFOSC image. Each ROI corresponds to one of the four polarimetric
# beams (0°, 90°, 45°, 135°). A mask is built to restrict source detection
# only to these regions, and the photutils segmentation algorithm identifies
# connected pixels above the given threshold. The resulting centroids define
# candidate target positions for photometric measurement in each beam.
# ---------------------------------------------------------------------------

def detect_sources_in_image(data_norm, threshold, npixels, roi_origins, roi_w, roi_h):
    """
    Detects sources inside the specified ROIs using photutils segmentation.
    Returns: ndarray of shape (N, 2) with centroids; if no sources, returns array of shape (0, 2).
    """
    from astropy.stats import sigma_clip
    # 1) Maschera ROI
    mask = np.zeros_like(data_norm, dtype=bool)
    for (rx, ry) in roi_origins:
        mask[ry:ry + roi_h, rx:rx + roi_w] = True
    data_masked = np.where(mask, data_norm, 0)
    # ===============================================================
    # 🔹 Rimozione del fondo colonnare con media (3σ clipping superiore)
    # ===============================================================
    data_corrected = data_masked.copy()

    n_cols = data_corrected.shape[1]
    for col in range(n_cols):
        col_data = data_corrected[:, col]

        # Considera solo pixel > 0
        col_valid = col_data[col_data > 0]

        if len(col_valid) >= 10:
            # Clipping solo sul lato superiore (elimina stelle)
            clipped = sigma_clip(col_valid, sigma_upper=3.0, sigma_lower=None, maxiters=5)
            sky_mean = np.mean(clipped.data[~clipped.mask])
        else:
            # Se pochi pixel validi, usa la mediana grezza
            sky_mean = np.median(col_valid) if len(col_valid) > 0 else 0.0

        # Sottrai il valore del cielo dalla colonna
        data_corrected[:, col] = col_data - sky_mean

    # 🔹 RIMOZIONE BANDA ORIZZONTALE (mediana per riga)
    row_med = np.where(mask, data_corrected, np.nan)
    row_med = np.nanmedian(row_med, axis=1)          # mediana su ciascuna riga
    data_corrected = data_masked - row_med[:, None]
    # Aggiorna immagine per la segmentazione

    data_masked = data_corrected

    # ===============================================================
    # 2) Segmentazione
    try:
        segm = detect_sources(data_masked, threshold, npixels=npixels)
    except Exception as e:
        print(f"[Error] detect_sources() failed: {e}")
        return np.empty((0, 2), dtype=float)

    # 3) Validazione segm
    if segm is None or getattr(segm, "data", None) is None or not np.any(segm.data > 0):
        print("[Info] No segments detected inside ROIs.")
        return np.empty((0, 2), dtype=float)

    '''# 4) Catalogo
    try:
        catalog = SourceCatalog(data_masked, segm)
    except Exception as e:
        print(f"[Error] Could not create SourceCatalog: {e}")
        return np.empty((0, 2), dtype=float) '''

    del data_corrected
    # 4) Catalogo
    try:
        catalog = SourceCatalog(data_masked, segm)
        del data_masked
        del segm
        gc.collect()
    except Exception as e:
        print(f"[Error] Could not create SourceCatalog: {e}")
        return np.empty((0, 2), dtype=float)

    '''# 🔹 FILTRO MORFOLOGICO ANTI-TRAIL
    MIN_AREA = 5          # px
    MAX_AREA = 400        # px  (aumenta se la PSF è molto larga)
    MAX_ELONG = 1.8       # rapporto assi; >2 tipico di scie/colonne
    SAT_LEVEL = 55000.0   # ADU circa; regola con il tuo gain/dinamica

    filtered = []
    for p in catalog:
        try:
            area = float(getattr(p, "area", np.nan))
            elong = float(getattr(p, "elongation", np.nan))
            peak  = float(getattr(p, "max_value", np.nan))
            x = float(getattr(p.xcentroid, "value", p.xcentroid))
            y = float(getattr(p.ycentroid, "value", p.ycentroid))

            if not (np.isfinite([area, elong, peak, x, y]).all()):
                continue
            if area < MIN_AREA or area > MAX_AREA:
                continue
            if elong > MAX_ELONG:
                continue
            # opzionale: scarta scie sature e molto allungate
            if peak > SAT_LEVEL and elong > 1.5:
                continue

            filtered.append((x, y))
        except Exception:
            continue

    coords = np.array(filtered, dtype=float).reshape(-1, 2)
    print(f"[Info] Detected {len(coords)} stellar-like sources after shape filtering.")'''

    # 5) Centroidi (x, y)
    coords = []
    for p in catalog:
        try:
            x = float(getattr(p.xcentroid, "value", p.xcentroid))
            y = float(getattr(p.ycentroid, "value", p.ycentroid))
            if np.isfinite(x) and np.isfinite(y):
                coords.append((x, y))
        except Exception:
            continue

    coords = np.array(coords, dtype=float).reshape(-1, 2)
    print(f"[Info] Detected {len(coords)} sources in defined ROIs.")

    # 🔹 Aggiunta: stampa dettagliata delle coordinate
    if len(coords) > 0:
        for i, (x, y) in enumerate(coords, start=1):
            print(f"  ROI {i}: x = {x:.2f}, y = {y:.2f}")
    else:
        print("  Nessuna sorgente rilevata.")
    return coords


# ---------------------------------------------------------------------------
# iterative_aperture_selection
# ---------------------------------------------------------------------------
# Determines the optimal aperture radius for flux extraction by iteratively
# enlarging the aperture until the relative change in enclosed flux is below
# a given fractional threshold (f_threshold). This approach avoids either
# underestimating or over-integrating the stellar flux. The routine also
# estimates the mean and RMS background level from an annulus surrounding
# the source, to be subtracted during flux computation.
# ---------------------------------------------------------------------------

def iterative_aperture_selection(data, x, y, f_threshold=0.01, max_iter=50):
    """
    Iteratively find the optimal aperture radius where the enclosed flux converges
    (relative change below f_threshold) while computing a robust local background.

    The iteration continues even if negative fluxes are encountered,
    keeping track of the last positive measurement.
    """

    scale_in_factor = 1.
    scale_out_factor = 2.
    r_ap = 2.0
    prev_flux = None
    last_valid = None  # memorizza ultima misura positiva
    sigma_clip = SigmaClip(sigma=3.0, maxiters=5)
    print(f"[DEBUG] In iterative: median={np.median(data):.2f}, min={data.min():.2f}, max={data.max():.2f}")

    for i in range(max_iter):
        # Definizione aperture e annulus
        aperture = CircularAperture((x, y), r=r_ap)
        annulus_ap = CircularAnnulus((x, y),
                                     r_in=scale_in_factor * r_ap,
                                     r_out=scale_out_factor * r_ap)
        annulus_mask = annulus_ap.to_mask(method='center')

        annulus_data = annulus_mask.multiply(data)
        annulus_data_1d = annulus_data[annulus_mask.data > 0]

        # Stima robusta del fondo
        if annulus_data_1d.size > 0 and np.isfinite(annulus_data_1d).any():
            clipped = sigma_clip(annulus_data_1d)
            valid = clipped[~clipped.mask]
            bkg_mean = np.mean(valid)
            bkg_rms = np.std(valid)
        else:
            bkg_mean, bkg_rms = 0.0, 0.0
            print(f"[Warning] No valid background pixels at ({x:.1f},{y:.1f})")

        # Fotometria
        phot_table = aperture_photometry(data, aperture)
        flux = phot_table['aperture_sum'][0] - bkg_mean * aperture.area

        # Controllo validità
        if not np.isfinite(flux):
            print(f"[Warning] Invalid flux (NaN/inf) at r={r_ap:.1f} for ({x:.1f},{y:.1f}); continuing.")
            r_ap += 1.0
            continue

        if flux <= 0:
            print(f"[Info] Negative flux ({flux:.2f}) at r={r_ap:.1f} for ({x:.1f},{y:.1f}); continuing. phot_table {phot_table['aperture_sum'][0]:.2f}, bkg_mean {bkg_mean:.2f}, aperture.area {aperture.area:.2f})")
            r_ap += 1.0
            continue

        # Flusso positivo → memorizza
        last_valid = (r_ap, scale_out_factor * r_ap, bkg_mean, bkg_rms)
        # Test di convergenza
        if prev_flux is not None:
            if abs(flux - prev_flux) / max(abs(flux), 1e-8) <= f_threshold:
                # print(f"[OK] Converged at r={r_ap:.1f} for ({x:.1f},{y:.1f})")
                return r_ap, scale_out_factor * r_ap, bkg_mean, bkg_rms

        prev_flux = flux
        r_ap += 1.0

    # Se non converge ma ci sono stati valori positivi → restituisci l’ultimo valido
    if last_valid is not None:
        print(f"[Info] No convergence but returning last positive flux at r={last_valid[0]:.1f} for ({x:.1f},{y:.1f})")
        return last_valid

    print(f"[Warning] No valid (positive) flux found for source at ({x:.1f},{y:.1f})")
    return None, None, None, None

# ---------------------------------------------------------------------------
# measure_fluxes_all_rois
# ---------------------------------------------------------------------------
# Performs aperture photometry for all four polarimetric ROIs using the
# positions of the selected target in each beam. For each ROI it measures:
#   - the integrated source flux corrected for background,
#   - the chosen aperture and background annulus radii,
#   - the local background mean and noise.
# Returns arrays of these quantities to be used in the Stokes computation.
# ---------------------------------------------------------------------------

def measure_fluxes_all_rois(data, selected_coords, f_threshold=0.01):
    """
    Measures flux and background stats for all selected sources (4 ROIs).
    Ensures consistent aperture and background annulus across all ROIs.
    Returns:
      fluxes: list of fluxes (I0, I90, I45, I135)
      aperture_radii: list of aperture radii
      annulus_radii: list of outer radii for background annulus
      bkg_means: list of mean background values
      bkg_rms: list of background RMS values (per pixel)
    """

    fluxes = []
    aperture_radii = []
    annulus_radii = []
    bkg_means = []
    bkg_rms = []
    refined_coords = []

    # --- STEP 1: misura preliminare per ciascuna ROI ---
    for i, (x, y) in enumerate(selected_coords):
        x_refined, y_refined = recenter_source(data, x, y, box=10)
        refined_coords.append((x_refined, y_refined))
        print(f"ROI {i} initial ({x:.1f},{y:.1f}) → refined ({x_refined:.1f},{y_refined:.1f})")

        r_ap, r_annulus_out, bkg_mean, bkg_std = iterative_aperture_selection(
            data, x_refined, y_refined, f_threshold=f_threshold
        )

        if r_ap is None:
            print(f"[Warning] Failed to find aperture radius for ROI {i} ({x:.1f},{y:.1f}) → skipping measurement.")
            return None, None, None, None, None

        aperture_radii.append(r_ap)
        annulus_radii.append(r_annulus_out)
        bkg_means.append(bkg_mean)
        bkg_rms.append(bkg_std)

    # --- STEP 2: determina raggi comuni ---
    r_ap_min = np.min(aperture_radii)
    r_ap_max = np.max(aperture_radii)
    r_annulus_min = np.min(annulus_radii)
    print(f"[INFO] Using common aperture radius = {r_ap_max:.2f} px, annulus outer radius = {r_annulus_min:.2f} px, minimum radii was {r_ap_min:.2f}")

    # --- STEP 3: ricalcola flussi con raggi comuni ---
    fluxes = []
    aperture_radii_common = []
    annulus_radii_common = []
    bkg_means_common = []
    bkg_rms_common = []

    for i, (x_refined, y_refined) in enumerate(refined_coords):
        # Garantisce geometria valida: r_out > r_in
        r_in = 1.5 * r_ap_max
        r_out = max(r_in + 1.0, r_annulus_min)  # forza r_out > r_in

        aperture = CircularAperture((x_refined, y_refined), r=r_ap_max)
        annulus_ap = CircularAnnulus(
            (x_refined, y_refined),
            r_in=r_in,
            r_out=r_out
        )

        # Estrai fondo locale coerente con la stessa geometria
        annulus_mask = annulus_ap.to_mask(method='center')
        annulus_data = annulus_mask.multiply(data)
        annulus_data_1d = annulus_data[annulus_mask.data > 0]

        if annulus_data_1d.size > 0 and np.isfinite(annulus_data_1d).any():
            clipped = SigmaClip(sigma=3.0, maxiters=5)(annulus_data_1d)
            valid = clipped[~clipped.mask]
            local_bkg_mean = np.mean(valid)
            local_bkg_std = np.std(valid)
        else:
            local_bkg_mean, local_bkg_std = 0.0, 0.0
            print(f"[Warning] No valid background pixels for ROI {i}.")

        # Fotometria coerente
        phot_table = aperture_photometry(data, aperture)
        flux = phot_table['aperture_sum'][0] - local_bkg_mean * aperture.area

        fluxes.append(flux)
        aperture_radii_common.append(r_ap_min)
        annulus_radii_common.append(r_annulus_min)
        bkg_means_common.append(local_bkg_mean)
        bkg_rms_common.append(local_bkg_std)

        print(f"ROI {i}: flux={flux:.1f}, Bkg={local_bkg_mean:.2f}, RMS={local_bkg_std:.2f}")

    return (
        fluxes,
        aperture_radii_common,
        annulus_radii_common,
        bkg_means_common,
        bkg_rms_common
    )
#---------------------------------------------------------------------------
# compute_stokes_parameters
# ---------------------------------------------------------------------------
# Computes the normalized Stokes parameters (Q, U) and the derived degree (P)
# and position angle (θ) of linear polarization, from the four measured
# intensities (I0, I90, I45, I135). This follows the standard formulae:
#   Q = (I0 - I90) / (I0 + I90)
#   U = (I45 - I135) / (I45 + I135)
#   P = sqrt(Q² + U²)
#   θ = 0.5 * arctan(U / Q)
# The function is central to the polarimetric analysis.
# ---------------------------------------------------------------------------
def compute_stokes_parameters(fluxes):
    I0, I90, I45, I135 = fluxes
    Q = (I0 - I90) / (I0 + I90)
    U = (I45 - I135) / (I45 + I135)
    P = np.sqrt(Q**2 + U**2)
    theta = 0.5 * np.degrees(np.arctan2(U, Q))
    return Q, U, P, theta
# ---------------------------------------------------------------------------
# match_source_in_new_image
# ---------------------------------------------------------------------------
# Tracks the same astrophysical source across successive FITS images,
# by matching its previously measured position with detections in a new frame.
# It uses a k-d tree nearest-neighbour search (cKDTree) to find the closest
# match within a maximum allowed displacement (max_move) inside each ROI.
# This enables automatic source tracking between exposures, avoiding repeated
# manual selection.
# ---------------------------------------------------------------------------

def match_source_in_new_image(prev_pos, catalog_coords, roi_origins, roi_w, roi_h, dx, gap, max_move=50):
    """
    Try to match the previously selected source position in a new image.
    Returns selected_coords if match is within max_move px, else None.
    """
    if prev_pos is None or len(prev_pos) == 0:
        return None # Cannot match if no previous position

    rx0, ry0 = roi_origins[0]
    inside_roi0_mask = (
        (catalog_coords[:,0] >= rx0) & (catalog_coords[:,0] <= rx0 + roi_w) &
        (catalog_coords[:,1] >= ry0) & (catalog_coords[:,1] <= ry0 + roi_h)
    )
    roi0_coords = catalog_coords[inside_roi0_mask]
    if len(roi0_coords) == 0:
        return None

    tree_roi0 = cKDTree(roi0_coords)
    # Ensure prev_pos[0] is a 2-element array
    if not isinstance(prev_pos, np.ndarray) or prev_pos.shape != (4, 2):
         return None # previous position is not in the expected format

    dist, idx = tree_roi0.query(prev_pos[0]) # prev_pos[0] is the coordinate in ROI 0
    if dist > max_move:
        return None

    base_source = roi0_coords[idx]
    selected_coords = [base_source]

    for i in range(1,4):
        rx, ry = roi_origins[i]
        if i < 2:
            x_shift = i * dx
        else:
            x_shift = i * dx + gap
        target_x = base_source[0] + x_shift
        target_y = base_source[1]

        inside_roi_mask = (
            (catalog_coords[:,0] >= rx) & (catalog_coords[:,0] <= rx + roi_w) &
            (catalog_coords[:,1] >= ry) & (catalog_coords[:,1] <= ry + roi_h)
        )
        roi_coords = catalog_coords[inside_roi_mask]
        if len(roi_coords) == 0:
            return None

        roi_tree = cKDTree(roi_coords)
        dist_i, idx_i = roi_tree.query([target_x, target_y], k=1)
        if dist_i > max_move:
            return None

        match_source = roi_coords[idx_i]
        selected_coords.append(match_source)

    return np.array(selected_coords) # Return as numpy array

def recenter_source(data, x0, y0, box=10):
        """
        Refine the centroid around an initial guess (x0, y0) using
        the center-of-mass method inside a small box.
        """
        x0, y0 = int(round(x0)), int(round(y0))
        y1, y2 = max(y0 - box, 0), min(y0 + box, data.shape[0])
        x1, x2 = max(x0 - box, 0), min(x0 + box, data.shape[1])
        sub = data[y1:y2, x1:x2]
        if sub.size == 0 or not np.isfinite(sub).any():
            return x0, y0  # fallback
        dy, dx = centroid_com(sub)
        return x1 + dx, y1 + dy
import numpy as np

def compute_stokes_dw(
    fluxes,
    gains=None,
    q_inst=0.0, u_inst=0.0,     # instrumental polarization to subtract (in Q,U)
    theta0_deg=0.0,             # angle zero-point (degrees) to rotate the vector (not used here)
    p_debias=True,              # Wardle & Kronberg debias
):
    """
    Stokes from a double-Wollaston single exposure (four beams: 0,90,45,135).

    Parameters
    ----------
    fluxes : iterable of 4 floats
        [I0, I90, I45, I135] after background subtraction (aperture photometry).
    gains : iterable of 4 floats or None
        Per-beam scalar corrections (flat/throughput): [g0, g90, g45, g135].
        If None, assumes unity gains.
    q_inst, u_inst : floats
        Instrumental polarization terms to subtract from (Q,U).
    theta0_deg : float
        Additive zero-point of the polarization angle (degrees).
    p_debias : bool
        If True, applies Wardle & Kronberg debias to P.

    Returns
    -------
    Q, U, P, theta_deg : floats
        Normalised Stokes, debiased degree of linear polarization, angle in degrees.
    """

    I0, I90, I45, I135 = map(float, fluxes)

    # Gains (throughput) correction — strongly recommended
    if gains is not None:
        g0, g90, g45, g135 = map(float, gains)
        if min(g0, g90, g45, g135) <= 0 or not np.isfinite([g0,g90,g45,g135]).all():
            raise ValueError("Invalid gains provided.")
        I0   /= g0
        I90  /= g90
        I45  /= g45
        I135 /= g135

    # Sanity checks
    if min(I0, I90, I45, I135) <= 0 or not np.isfinite([I0,I90,I45,I135]).all():
        raise ValueError("Non-positive or non-finite beam flux(es). Check centring, background, flats.")

    denom_Q = I0 + I90
    denom_U = I45 + I135
    if denom_Q <= 0 or denom_U <= 0:
        raise ValueError("Non-positive denominators in Stokes computation.")

    # Stokes (ideal four-channel)
    Q = (I0 - I90) / denom_Q
    U = (I45 - I135) / denom_U

    # Subtract instrumental polarization (measured on unpolarised star)
    Q -= q_inst
    U -= u_inst

    # Rotate by zero-point (convert instrument frame -> celestial frame)
    if theta0_deg != 0.0:
        phi = np.radians(2.0 * theta0_deg)  # note the factor 2 on Stokes space
        Q, U = (Q*np.cos(phi) - U*np.sin(phi),
                Q*np.sin(phi) + U*np.cos(phi))

    # Degree and angle
    P = np.hypot(Q, U)
    theta_deg = 0.5 * np.degrees(np.arctan2(U, Q))

    # Debias (Wardle & Kronberg): P_db = sqrt(P^2 - sigma_P^2), if known
    # If you have per-beam photometric errors, propagate to sigma_Q, sigma_U, hence sigma_P.
    if p_debias:
        # Minimal conservative heuristic (replace with proper error propagation):
        # assume sigma_P ~ max(1/sqrt(I0), ...). Better: compute from aperture variances.
        # Here we avoid negative under the sqrt by clipping:
        sigma_P = 0.0  # <- set your propagated value here
        P = np.sqrt(max(P*P - sigma_P*sigma_P, 0.0))
    # ===============================================================
    # Calcolo incertezze su Q, U e P
    # ===============================================================
    I0_err, I45_err, I90_err, I135_err = np.sqrt(fluxes)
    # Error propagation per Q
    dQ_dI0  =  2 * I90 / (I0 + I90)**2
    dQ_dI90 = -2 * I0  / (I0 + I90)**2
    sigma_Q = np.sqrt((dQ_dI0 * I0_err)**2 + (dQ_dI90 * I90_err)**2)

    # Error propagation per U
    dU_dI45  =  2 * I135 / (I45 + I135)**2
    dU_dI135 = -2 * I45  / (I45 + I135)**2
    sigma_U  = np.sqrt((dU_dI45 * I45_err)**2 + (dU_dI135 * I135_err)**2)

    # Propagazione su P = sqrt(Q² + U²)
    sigma_P = np.sqrt((Q * sigma_Q)**2 + (U * sigma_U)**2) / np.sqrt(Q**2 + U**2)

    # Ritorna tutto insieme
    return Q, U, P, theta_deg, sigma_Q, sigma_U, sigma_P

# ---------------------------------------------------------------------------
# class InteractiveSourceSelector
# ---------------------------------------------------------------------------
# Interactive widget-driven interface built with ipywidgets, managing:
#   - manual source selection for the first image (or when auto-tracking fails),
#   - automated tracking and flux measurement for subsequent images,
#   - on-screen display of images, ROIs, detected sources, and user options.
#
# Key methods:
#   _log(): writes messages to the log panel.
#   _on_dropdown_change(): enables user selection of a detected source.
#   _on_continue_button_click(): triggers photometric measurement after user validation.
#   _derive_all_roi_coords(): computes expected beam positions from one ROI to others.
#   plot_image_with_sources(): displays image, detected sources, and selected targets.
#   _process_current_file(): orchestrates loading, source detection, and measurement per image.
#   _measure_and_save_results(): computes fluxes and Stokes parameters for the selected source.
#   _save_results(): writes the accumulated measurements to an ASCII table.
# The class provides both interactivity and automation for full sequence processing.
# ---------------------------------------------------------------------------
class InteractiveSourceSelector:
    def __init__(self, file_list, roi_origins, roi_w, roi_h, dx, gap, sigma_threshold, npixels, max_move, snr_threshold):
        self.file_list = file_list
        self.roi_origins = roi_origins
        self.roi_w = roi_w
        self.roi_h = roi_h
        self.dx = dx
        self.gap = gap
        self.sigma_threshold = sigma_threshold
        self.npixels = npixels
        self.max_move = max_move
        self.snr_threshold = snr_threshold

        self.current_file_index = 0
        self.prev_selected_coords = np.array([]) # Initialize as empty numpy array
        self.results = []
        self._selected_coords = None
        self._done = False # This flag might become less critical if flow is fully button-driven

        # Widgets
        self.progress_label = widgets.Label(f'Processing file {self.current_file_index + 1}/{len(self.file_list)}: {os.path.basename(self.file_list[self.current_file_index])}')
        self.output_area = widgets.Output() # For plots and dynamic messages
        self.source_dropdown = widgets.Dropdown(description='Select source:', layout=widgets.Layout(width='400px'))
        self.continue_button = widgets.Button(description='CONTINUE', button_style='success', disabled=True)
        self.selected_source_label = widgets.Label('Selected source: None', layout=widgets.Layout(width='150px'))
        self.log_output = widgets.Output(layout={'border': '1px solid black', 'height': '150px', 'overflow_y': 'auto'}) # For log messages

        # Link widgets
        self.source_dropdown.observe(self._on_dropdown_change, names='value')
        self.continue_button.on_click(self._on_continue_button_click)

        # Layout
        self.interface = widgets.VBox([
            self.progress_label,
            self.output_area,
            widgets.HBox([self.source_dropdown, self.continue_button, self.selected_source_label]),
            self.log_output
        ])

    def _log(self, message):
         """Helper to print messages to the log output area."""
         with self.log_output:
             print(message)
    def _as_float(self, x, default=None):
            """Return a Python float if possible, else `default`."""
            try:
                # unwrap numpy scalars/arrays
                if hasattr(x, "value"):  # astropy Quantity
                    x = x.value
                if isinstance(x, (list, tuple, np.ndarray)) and np.size(x) == 1:
                    x = np.asarray(x).reshape(()).item()
                return float(x)
            except Exception:
                return default

    def _as_str(self, x, default=""):
            """Return a clean string (no bytes), stripped; fallback to `default`."""
            try:
                if isinstance(x, bytes):
                    x = x.decode("utf-8", errors="ignore")
                s = str(x).strip()
                return s
            except Exception:
                return default


    def _to_degrees(val):
            try:
                v = float(val)
                return np.degrees(v)
            except Exception:
                return val  # keep as-is if non-numeric


    def _on_dropdown_change(self, change):
        """Enables the continue button when a source is selected."""
        selected_index = change['new']
        if selected_index is not None and hasattr(self, '_current_roi0_sources') and self._current_roi0_sources is not None and selected_index < len(self._current_roi0_sources):
            selected_source = self._current_roi0_sources[selected_index]
            self.selected_source_label.value = f'Selected source: ({selected_source[0]:.1f}, {selected_source[1]:.1f})'
            self.continue_button.disabled = False
        else:
            self.selected_source_label.value = 'Selected source: None'
            self.continue_button.disabled = True

    def _on_continue_button_click(self, b):
        """Handles the button click: processes the current file and proceeds."""
        with self.output_area:
            clear_output(wait=True) # Clear the plot and previous selection interface

        selected_index = self.source_dropdown.value
        if selected_index is not None and hasattr(self, '_current_roi0_sources') and self._current_roi0_sources is not None and selected_index < len(self._current_roi0_sources):
             base_source = self._current_roi0_sources[selected_index]
             selected_coords = self._derive_all_roi_coords(base_source)
             self._selected_coords = selected_coords # Store the selected coords

             # Log the manually selected source ID
             self._log(f"Manually selected source ID: {selected_index}")

             # Proceed to process the current file with the selected coordinates
             if self.current_file_index >= len(self.file_list):
                  self._log("All files processed. Nothing more to measure.")
                  return
             filename = self.file_list[self.current_file_index]

             try:
                 hdul = fits.open(filename)
                 data = hdul[0].data.astype(float)
                 hdul.close()
                 del hdul
                 self._measure_and_save_results(data, selected_coords)
                 gc.collect() # Garbage collection
                 self.prev_selected_coords = selected_coords # Update for next iteration
                 self.current_file_index += 1
                 self._process_current_file() # Process the next file automatically
             except Exception as e:
                 self._log(f"Error processing file after selection {os.path.basename(filename)}: {e}")
                 # Decide how to handle error: skip or require re-selection?
                 # For now, skip and move to next file.
                 self.current_file_index += 1
                 self._process_current_file()
        else:
             self._log("Error: Continue pressed without a valid source selected.")
             # Decide how to handle error: user didn't select or no sources available?
             # For now, skip and move to next file.
             self.current_file_index += 1
             self._process_current_file()

    def _derive_all_roi_coords(self, base_source):
        """Derives the expected coordinates of the source in other ROIs."""
        derived_coords = [base_source]
        for i in range(1, 4):
            rx, ry = self.roi_origins[i]
            if i < 2:
                x_shift = i * self.dx
            else:
                x_shift = i * self.dx + self.gap
            target_x = base_source[0] + x_shift
            target_y = base_source[1]
            derived_coords.append((target_x, target_y))
        return np.array(derived_coords)


    def plot_image_with_sources(self, data_norm, catalog_coords, selected_coords=None):
        """Plots the image, ROIs, detected sources, and optionally selected sources."""
        with self.output_area:
            clear_output(wait=True) # Clear previous plot before displaying new one
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.imshow(data_norm, origin='lower', cmap='gray',
                      vmin=np.percentile(data_norm, 5), vmax=np.percentile(data_norm, 99))
            for i, (rx, ry) in enumerate(self.roi_origins):
                ax.add_patch(patches.Rectangle((rx, ry), self.roi_w, self.roi_h,
                               edgecolor='red', facecolor='none', linewidth=1.5))
                ax.text(rx + 5, ry + self.roi_h - 15, f'ROI {i}', color='red')

            # Plot all sources
            for x, y in catalog_coords:
                ax.plot(x, y, 'o', color='lime', markersize=5)

            # Plot selected sources if available
            if selected_coords is not None and selected_coords.shape == (4, 2):
                 for i, (x, y) in enumerate(selected_coords):
                      ax.plot(x, y, 'X', color='cyan', markersize=10, label=f'Selected (ROI {i})')
                 ax.legend()
            # --- NEW: show aperture and annulus sizes ---
            if selected_coords is not None and hasattr(self, 'results') and len(self.results) > 0:
                last_result = self.results[-1]
                ap_radii = last_result.get('aperture_radii', [])
                ann_radii = last_result.get('annulus_radii', [])
                for i, (x, y) in enumerate(selected_coords):
                    if i < len(ap_radii):
                        circ = patches.Circle((x, y), ap_radii[i], edgecolor='cyan',
                                              facecolor='none', linestyle='--', linewidth=1)
                        ann = patches.Circle((x, y), ann_radii[i], edgecolor='orange',
                                            facecolor='none', linestyle=':', linewidth=1)
                        ax.add_patch(circ)
                        ax.add_patch(ann)
            # Find sources inside ROI 0 (only for manual selection plotting)
            if selected_coords is None:
                rx0, ry0 = self.roi_origins[0]
                roi0_mask = (
                    (catalog_coords[:,0] >= rx0) & (catalog_coords[:,0] <= rx0 + self.roi_w) &
                    (catalog_coords[:,1] >= ry0) & (catalog_coords[:,1] <= ry0 + self.roi_h)
                )
                roi0_sources = catalog_coords[roi0_mask]
                self._current_roi0_sources = roi0_sources # Store for dropdown

                if len(roi0_sources) == 0:
                    self._log("No detected sources inside ROI 0 to select from.")
                    self.source_dropdown.options = []
                    self.continue_button.disabled = True
                else:
                    dropdown_options = [(f"{idx}: x={x:.1f}, y={y:.1f}", idx) for idx, (x, y) in enumerate(roi0_sources)]
                    self.source_dropdown.options = dropdown_options
                    self.source_dropdown.value = None # Reset value to trigger observe if same source is selected again
                    self.continue_button.disabled = True # Disable until a source is selected

                    # Label each source inside ROI 0 with index
                    for idx, (x, y) in enumerate(roi0_sources):
                        ax.text(x+3, y, f"{idx}", color='yellow', fontsize=10, weight='bold')
                ax.set_title("Detected sources with indices inside ROI 0 (Manual Selection)")
            else:
                 ax.set_title("Automatically Selected Sources")


            display(fig) # Display the plot
            plt.close(fig) # Close the figure to prevent duplicate display

    def show(self):
        """Displays the interactive interface."""
        # No need to manage _done flag explicitly for a button-driven flow
        # The display of the interface itself pauses the process until the button is clicked
        display(self.interface)

    def is_done(self):
        """This method is less relevant in a button-driven flow."""
        # Kept for compatibility if needed, but the flow is now controlled by _on_continue_button_click
        return self._done

    def get_selected_coords(self):
        """Returns the selected source coordinates or None."""
        # This is now typically called within the _on_continue_button_click method
        return self._selected_coords

    def update_progress(self, file_index):
        """Updates the progress label."""
        self.current_file_index = file_index
        if self.current_file_index < len(self.file_list):
            self.progress_label.value = f'Processing file {self.current_file_index + 1}/{len(self.file_list)}: {os.path.basename(self.file_list[self.current_file_index])}'
        else:
             self.progress_label.value = 'Processing complete.'


    def run(self):
        """Starts the file processing sequence."""
        # This method initiates the processing of the first file.
        self._process_current_file()

    def _process_current_file(self):
        """Processes the current FITS file, handling automatic or manual selection."""
        from astropy.stats import SigmaClip

        # ===============================================================
        # 🔹 0) Check for completion
        # ===============================================================
        if self.current_file_index >= len(self.file_list):
            self._log("All files processed.")
            self._save_results()
            self.progress_label.value = 'Processing complete.'
            return

        # ===============================================================
        # 🔹 1) Load FITS image
        # ===============================================================
        filename = self.file_list[self.current_file_index]
        self.update_progress(self.current_file_index)
        self._log(f"\nProcessing {os.path.basename(filename)}")

        try:
            with fits.open(filename) as hdul:
                data = hdul[0].data.astype(float)
        except Exception as e:
            self._log(f"[Error] Could not read FITS file {os.path.basename(filename)}: {e}")
            self.current_file_index += 1
            self._process_current_file()  # Skip to next file
            return

        # ===============================================================
        # 🔹 2) Ritaglia la parte superiore (solo regione utile alla polarimetria)
        # ===============================================================
        # Mantieni solo la zona superiore di altezza pari alla ROI (come richiesto)
        data_reduced = data[:self.roi_h, :]

        # ===============================================================
        # 🔹 3) Rimozione del fondo colonnare con media (3σ clipping superiore)
        # ===============================================================
        clipper = SigmaClip(sigma_upper=3.0, sigma_lower=None, maxiters=5)
        data_corrected = data_reduced.copy()


        for col in range(data_corrected.shape[1]):
            col_data = data_corrected[:, col]

            # Considera solo valori finiti e non negativi
            col_valid = col_data[np.isfinite(col_data) & (col_data > 0)]
            if len(col_valid) >= 10:
                clipped = clipper(col_valid)
                sky_mean = np.mean(clipped.data[~clipped.mask])
            elif len(col_valid) > 0:
                sky_mean = np.median(col_valid)
            else:
                sky_mean = 0.0

            # Sottrai il valore medio del cielo per colonna
            data_corrected[:, col] = col_data - sky_mean

        data_reduced = data_corrected
        data = data_reduced
        # ===============================================================
        # 🔹 4) Normalizzazione per il rilevamento
        # ===============================================================
        rx0, ry0 = self.roi_origins[0]
        data_roi0 = data_reduced[ry0:ry0 + self.roi_h, rx0:rx0 + self.roi_w]
        median_roi0 = np.median(data_roi0)
        std_roi0 = np.std(data_roi0)
        data_norm = (data_reduced - median_roi0) / (std_roi0 if std_roi0 > 0 else 1.0)

        # ===============================================================
        # 🔹 5) Rilevamento sorgenti
        # ===============================================================
        catalog_coords = detect_sources_in_image(
            data_norm, self.sigma_threshold, self.npixels,
            self.roi_origins, self.roi_w, self.roi_h
        )

        # Libera memoria intermedia
        del data_corrected
        gc.collect()

        # ===============================================================
        # 🔹 6) Selezione automatica / manuale
        # ===============================================================

        # --- Filtra le sorgenti nella ROI 0
        rx0, ry0 = self.roi_origins[0]
        roi0_mask = (
            (catalog_coords[:, 0] >= rx0) & (catalog_coords[:, 0] <= rx0 + self.roi_w) &
            (catalog_coords[:, 1] >= ry0) & (catalog_coords[:, 1] <= ry0 + self.roi_h)
        )
        roi0_sources = catalog_coords[roi0_mask]

        # --- Caso 1: tracking attivo e sorgente vicina alla posizione precedente
        if (
            self.prev_selected_coords is not None
            and self.prev_selected_coords.shape == (4, 2)
            and len(roi0_sources) > 0
        ):
            prev_x, prev_y = self.prev_selected_coords[0]
            distances = np.sqrt((roi0_sources[:, 0] - prev_x)**2 + (roi0_sources[:, 1] - prev_y)**2)
            close_mask = distances < 25.0
            close_sources = roi0_sources[close_mask]
            if len(close_sources) == 1:
                matched = close_sources[0]
                self._log(f"Source found within 25 px of previous position → auto-selected at "
                          f"({matched[0]:.1f}, {matched[1]:.1f})")

                selected_coords = self._derive_all_roi_coords(matched)
                self._selected_coords = selected_coords
                self._measure_and_save_results(data, selected_coords)
                self.prev_selected_coords = selected_coords
                self.current_file_index += 1
                self._process_current_file()
                del data, catalog_coords
                gc.collect()
                return
            elif len(close_sources) > 1:
                self._log("[WARNING] Multiple sources within 25 px of previous position → switching to manual selection.")

        # --- Caso 2: nessuna sorgente trovata
        if len(roi0_sources) == 0:
            base = os.path.basename(filename)
            self._log(f"[Warning] No sources detected in ROI 0 for {base}. Moving to 'faint_sources' and skipping file.")
            try:
                faint_dir = os.path.join(os.path.dirname(filename), "faint_sources")
                os.makedirs(faint_dir, exist_ok=True)
                dest_path = os.path.join(faint_dir, base)
                if os.path.exists(dest_path):
                    name, ext = os.path.splitext(base)
                    dest_path = os.path.join(faint_dir, f"{name}_dup{ext}")
                shutil.move(filename, dest_path)
                self._log(f"→ File moved to {dest_path}")
            except Exception as move_err:
                self._log(f"[Error] Could not move {base} to faint_sources: {move_err}")
            self.current_file_index += 1
            self._process_current_file()
            return

        # --- Caso 3: singola sorgente → auto-selection
        if len(roi0_sources) == 1:
            single_source = roi0_sources[0]
            print(f"Single source detected, auto-selected at ({single_source[0]:.1f}, {single_source[1]:.1f})")
            selected_coords = self._derive_all_roi_coords(single_source)
            self._selected_coords = selected_coords
            self._measure_and_save_results(data, selected_coords)
            self.prev_selected_coords = selected_coords
            self.current_file_index += 1
            self._process_current_file()
            return

        # --- Caso 4: primo frame o fallimento tracking → selezione manuale
        if (
            self.current_file_index == 0
            or self.prev_selected_coords is None
            or self.prev_selected_coords.shape != (4, 2)
        ):
            if catalog_coords is None or len(catalog_coords) == 0:
                base = os.path.basename(filename)
                self._log(f"[Warning] No sources found for first frame {base}. "
                          "Moving to 'faint_sources' and skipping manual selection.")
                try:
                    faint_dir = os.path.join(os.path.dirname(filename), "faint_sources")
                    os.makedirs(faint_dir, exist_ok=True)
                    dest_path = os.path.join(faint_dir, base)
                    if os.path.exists(dest_path):
                        name, ext = os.path.splitext(base)
                        dest_path = os.path.join(faint_dir, f"{name}_dup{ext}")
                    shutil.move(filename, dest_path)
                    self._log(f"→ File moved to {dest_path}")
                except Exception as move_err:
                    self._log(f"[Error] Could not move {base} to faint_sources: {move_err}")
                gc.collect()
                self.current_file_index += 1
                self._process_current_file()
                return

            self._log("Manual selection required for the first image or due to tracking failure.")
            self.plot_image_with_sources(data_norm, catalog_coords)
            self.show()
            return

        # --- Caso 5: tracking automatico
        selected_coords = match_source_in_new_image(
            self.prev_selected_coords,
            catalog_coords, self.roi_origins,
            self.roi_w, self.roi_h,
            self.dx, self.gap, self.max_move
        )

        if selected_coords is None:
            self._log("Automatic matching failed, switching to manual selection.")
            self.plot_image_with_sources(data_norm, catalog_coords)
            self.show()
        else:
            self._log("Automatic matching successful.")
            self._selected_coords = selected_coords
            self.plot_image_with_sources(data_norm, catalog_coords, selected_coords=selected_coords)

            # Identifica ID sorgente selezionata
            roi0_sources = catalog_coords[roi0_mask]
            if len(roi0_sources) > 0:
                try:
                    match_idx = np.where(
                        np.isclose(roi0_sources, selected_coords[0], atol=0.1).all(axis=1)
                    )[0][0]
                    self._log(f"Automatically selected source ID (in ROI 0 catalog): {match_idx}")
                except IndexError:
                    self._log("Could not determine source ID for automatically selected source.")

            self._measure_and_save_results(data, selected_coords)
            self.prev_selected_coords = selected_coords
            self.current_file_index += 1
            self._process_current_file()

    def _measure_and_save_results(self, data, selected_coords):
        """Measures fluxes and appends results, including FITS metadata."""
        if selected_coords is None or selected_coords.shape != (4, 2):
            self._log("[WARNING] Cannot measure fluxes: invalid or missing selected coordinates.")
            return

        # Recupera nome file corrente
        filename = self.file_list[self.current_file_index]
        # --- Leggi header FITS ---
        with fits.open(filename) as hdul:
            hdr = hdul[0].header

        obj_name = self._as_str(hdr.get("OBJECT", ""))

        rot_angle = hdr.get("ROTANGLE", None)
        rotangle_rad = self._as_float(rot_angle, default=None)

        date_obs_raw = hdr.get("DATE-OBS", "")
        date_obs = self._as_str(date_obs_raw)  # keep ISO-like text

        alt_obs = self._as_float(hdr.get("ALTITUDE", None), default=np.nan)
        azi_obs = self._as_float(hdr.get("AZIMUTH",  None), default=np.nan)

        # EXPTIME robust
        exptime = self._as_float(hdr.get("EXPTIME", None), default=np.nan)

        # FILTER robust
        filt = (hdr.get("FILTER") or hdr.get("FILTER1") or hdr.get("FILT") or "")
        filt = self._as_str(filt)

        if rotangle_rad is not None and np.isfinite(rotangle_rad):
           rotangle_deg = np.degrees(rotangle_rad)
        else:
           rotangle_deg = ""  # empty cell rather than an array or bad string

        # --- Misura i flussi ---
        fluxes, ap_radii, ann_radii, bkg_means, bkg_stds = measure_fluxes_all_rois(
            data, selected_coords, f_threshold=0.01
        )

        if fluxes is None:
            self._log("[WARNING] Skipping image due to failed aperture measurement.")
            return

        I0, I90, I45, I135 = fluxes
        print(f"\nFluxes for {os.path.basename(filename)}:")
        print(f"   I0={I0:.2f}, I90={I90:.2f}, I45={I45:.2f}, I135={I135:.2f}")

        try:
            Q, U, P, theta,sigma_Q,sigma_U,sigma_P, = compute_stokes_dw(fluxes)
            self._log(f"Measured Q={Q:.3f}, U={U:.3f}, P={P:.3f}, theta={theta:.3f}")

            if np.isfinite(exptime) and exptime > 0:
                fluxes_per_sec = (np.array(fluxes, dtype=float) / exptime).tolist()
            else:
                fluxes_per_sec = [np.nan, np.nan, np.nan, np.nan]
            self.results.append({
                'filename': os.path.basename(filename),
                'object':   obj_name,
                'date_obs': date_obs,
                'filter':   filt,
                'rotangle_deg': self._as_float(rotangle_deg, default=np.nan),
                'Q': float(Q), 'U': float(U), 'P': float(P), 'theta': float(theta),
                'fluxes': fluxes_per_sec,
                'azi_obs': azi_obs, 'alt_obs': alt_obs,
                'bkg_means': [float(x) for x in bkg_means],
                'bkg_stds':  [float(x) for x in bkg_stds],
                'aperture_radii': [float(x) for x in ap_radii],
                'annulus_radii':  [float(x) for x in ann_radii],
                'Q_err': float(sigma_Q), 'U_err': float(sigma_U), 'P_err': float(sigma_P),
            })


        except Exception as e:
            self._log(f"Error computing Stokes parameters for {filename}: {e}")

    def _save_results(self):
        """Save all accumulated results to a CSV table."""
        print("saving results")
        if not self.results:
            self._log("No results to save.")
            return

        import csv
        folder_path = os.path.dirname(self.file_list[0])
        out_csv = os.path.join(folder_path, "polarimetry_summary.csv")

        with open(out_csv, "w", newline="") as f:
            writer = csv.writer(f, delimiter=",")
    # Intestazione aggiornata ▼
            writer.writerow([
                "filename", "object", "date_obs", "filter",
                "rotangle_deg", "Q", "U", "P", "theta",
                "Q_err", "U_err", "P_err",
                "I_sum",                 # <--- nuovo campo
                "azimuth", "altitude"
            ])

            for r in self.results:
                # Calcolo sicuro della somma dei flussi, se disponibili
                I_sum = None
                if isinstance(r.get("fluxes", None), (list, tuple)) and len(r["fluxes"]) >= 4:
                    I_sum = sum(r["fluxes"][:4])
                else:
                    I_sum = 0.0

                writer.writerow([
                    r.get("filename", ""),
                    r.get("object", ""),
                    r.get("date_obs", ""),
                    r.get("filter", ""),
                    f"{float(r.get('rotangle_deg', 0)):.6f}" if isinstance(r.get("rotangle_deg", None), (int, float)) else str(r.get("rotangle_deg", "")),
                    f"{r.get('Q', 0):.6f}",
                    f"{r.get('U', 0):.6f}",
                    f"{r.get('P', 0):.6f}",
                    f"{r.get('theta', 0):.3f}",
                    f"{r.get('Q_err', 0):.6f}",
                    f"{r.get('U_err', 0):.6f}",
                    f"{r.get('P_err', 0):.6f}",
                    f"{I_sum:.3f}",         # <--- somma dei flussi
                    f"{r.get('azi_obs', 0):.3f}",
                    f"{r.get('alt_obs', 0):.3f}",
                ])
            self._log(f"\nSummary CSV saved to {out_csv}")

# ---------------------------------------------------------------------------
# process_all_files_with_interface
# ---------------------------------------------------------------------------
# Main entry point to execute the full polarimetric reduction pipeline.
# Scans a specified directory for FITS files, instantiates the interactive
# selector class, and initiates the iterative processing of all images.
# The workflow alternates between automatic and manual selection depending
# on the tracking success, ultimately generating a table of Stokes parameters
# and polarization angles for all frames in the sequence.
# ---------------------------------------------------------------------------
def process_all_files_with_interface(folder_path, roi_origins, roi_w, roi_h,
                                     dx, gap, sigma_threshold, npixels,
                                     max_move, snr_threshold):
    """
    Scansiona la cartella indicata, elabora tutti i file FITS e FITS.GZ,
    e avvia l'interfaccia interattiva per la selezione delle sorgenti.
    """

    # --- 1️⃣ Raccoglie i file presenti nella cartella ---
    all_files = os.listdir(folder_path)
    file_list = sorted([
        os.path.join(folder_path, f)
        for f in all_files
        if f.lower().endswith(".fits") or f.lower().endswith(".fits.gz")
    ])

    # --- 2️⃣ Controllo dei file trovati ---
    if not file_list:
        print(f"No FITS or FITS.GZ files found in {folder_path}")
        return

    print(f"Found {len(file_list)} FITS/FITS.GZ files to process:")
    for f in file_list[:10]:  # stampa solo i primi 10 per non saturare l’output
        print("  ", os.path.basename(f))
    if len(file_list) > 10:
        print(f"  ... and {len(file_list) - 10} more files.\n")

    '''# --- 3️⃣ Avvia l'interfaccia interattiva ---
    try:
        selector = InteractiveSourceSelector(
            file_list, roi_origins, roi_w, roi_h,
            dx, gap, sigma_threshold, npixels, max_move, snr_threshold
        )
        selector.show()
    except Exception as e:
        print(f"Error launching interactive processing: {e}")'''

    # Instantiate the selector with all necessary parameters
    selector = InteractiveSourceSelector(
        file_list, roi_origins, roi_w, roi_h,
        dx, gap,  sigma_threshold, npixels, max_move, snr_threshold
    )
    # Run the selector; it will manage the file processing and interaction flow
    selector.run()

# --- Cell S1z2OfQfWQrj (parameters and call) needs to be updated to call the non-async function ---
# Assuming the parameter definitions exist from cell S1z2OfQfWQrj
# sigma_threshold, npixels, roi_w, roi_h, roi0_x, roi0_y, dx, gap, roi_origins, max_move, snr_threshold, folder_path
# are assumed to be defined before calling this function.

# Example call (would be in a separate cell, e.g., the modified S1z2OfQfWQrj)
# process_all_files_with_interface(folder_path, roi_origins, roi_w, roi_h, dx, gap, sigma_threshold, npixels, max_move, snr_threshold)

# Note: The interactive interface will be displayed inline when selector.show() is called.
# The flow pauses at that point until the user clicks the 'CONTINUE' button.
# The processing of the next file is triggered by the button click handler.

In [ ]:
import shutil
from astropy.io import fits
import os
import glob
import numpy as np

# HOTBIRD 13 F
input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251025/HOTBIRD_13F_PL"
#input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251025/HOTBIRD"

# UNPOLARIZED STAR
input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251026/HD39587"

# POLARIZED STAR
input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251027/HD283809"

# POLARIZED STAR
input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251028/POL_STARS2"

input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251028/POL_STARS"

# UNPOLARIZED STAR
input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251028/HD42182_1st"
#input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251028/HD42182_2nd"
#input_folder = "/content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251028/HD42182_Test"





output_folder = input_folder + "/non_pol"

os.makedirs(output_folder, exist_ok=True)
for filename in os.listdir(input_folder):
    # accetta estensioni .fits oppure .fits.gz
    lower = filename.lower()
    if not (lower.endswith(".fits") or lower.endswith(".fits.gz")):
        continue

    filepath = os.path.join(input_folder, filename)

    with fits.open(filepath) as hdul:
        hdr = hdul[0].header

        grat = hdr.get("GRAT_TYP", None)
        obj = hdr.get("OBJECT", "N/A")
        exptime = hdr.get("EXPTIME", "N/A")
        filt = hdr.get("FILTER", "N/A")
        rotangle = hdr.get("ROTANGLE", "N/A")
        date_obs = hdr.get("DATE-OBS", "N/A")
        alt_obs = hdr.get("ALTITUDE", "N/A")
        azi_obs = hdr.get("AZIMUTH", "N/A")
        hdul.close()
        del hdul
        gc.collect()


        if isinstance(rotangle, (int, float)):
            rotangle_deg = np.degrees(rotangle)
            rotangle_str = f"{rotangle_deg:.1f}"
        else:
            rotangle_str = str(rotangle)

        if grat != "Pol":
            print(f"Moving {filename} (Object: {obj}, Exptime: {exptime}, Filter: {filt}, RotAngle: {rotangle_str} deg, Date-Obs: {date_obs}) to {output_folder}")
            shutil.move(filepath, os.path.join(output_folder, filename))
        else:
            print(f"Keeping {filename} (Object: {obj}, Exptime: {exptime}, Filter: {filt}, RotAngle: {rotangle_str} deg, Date-Obs: {date_obs})")


Keeping AF969610.fits.gz (Object: HD42182, Exptime: 1.0, Filter: r-Sloan, RotAngle: -0.0 deg, Date-Obs: 2025-10-29T01:09:43.9)
Keeping AF969611.fits.gz (Object: HD42182, Exptime: 1.0, Filter: V-Bessel, RotAngle: -0.0 deg, Date-Obs: 2025-10-29T01:10:32.7)
Keeping AF969612.fits.gz (Object: HD42182, Exptime: 1.0, Filter: NONE, RotAngle: -0.0 deg, Date-Obs: 2025-10-29T01:11:10.5)
Keeping AF969614.fits.gz (Object: HD42182, Exptime: 3.0, Filter: i-Sloan, RotAngle: -0.0 deg, Date-Obs: 2025-10-29T01:12:30.3)
Keeping AF969613.fits.gz (Object: HD42182, Exptime: 10.0, Filter: B-Bessel, RotAngle: -0.0 deg, Date-Obs: 2025-10-29T01:11:44.0)
Keeping AF969615.fits.gz (Object: HD42182, Exptime: 1.0, Filter: r-Sloan, RotAngle: -0.0 deg, Date-Obs: 2025-10-29T01:13:53.5)
Keeping AF969616.fits.gz (Object: HD42182, Exptime: 1.0, Filter: r-Sloan, RotAngle: -0.0 deg, Date-Obs: 2025-10-29T01:14:00.9)
Keeping AF969617.fits.gz (Object: HD42182, Exptime: 1.0, Filter: r-Sloan, RotAngle: -0.0 deg, Date-Obs: 2025-10

In [ ]:
# === PARAMETERS ===
sigma_threshold = 3.0
npixels = 5
max_move = 5   # max px for automatic source tracking between images
snr_threshold = 2  # for aperture radius selection
#####################

folder_path = input_folder  # esempio: "/content/.../HD283809"

# --- 1️⃣ Elenco dei file nella directory ---
all_files = os.listdir(folder_path)
file_list = sorted([
    os.path.join(folder_path, f)
    for f in all_files
    if f.lower().endswith(".fits") or f.lower().endswith(".fits.gz")
])

# --- 2️⃣ Controllo ---
if not file_list:
    print(f"[Warning] No FITS or FITS.GZ files found in {folder_path}")
else:
    n_files = len(file_list)
    print(f"[INFO] Found {n_files} FITS/FITS.GZ files in {folder_path}")

    # mostra solo i primi 10 file, per evitare output lunghissimo
    for f in file_list[:10]:
        print("   ", os.path.basename(f))
    if n_files > 10:
        print(f"   ... and {n_files - 10} more files not shown.\n")


    # --- 3️⃣ Leggi il primo file per determinare il binning ---
    try:
        first_file = file_list[0]
        print(f"\nReading header from: {os.path.basename(first_file)}")

        with fits.open(first_file) as hdul:
            hdr = hdul[0].header
            binx = hdr.get("BINX", 1)
            biny = hdr.get("BINY", 1)
        del hdul
        gc.collect()

        print(f"Binning detected: {binx}x{biny}")

        # --- 4️⃣ Parametri per ROI a seconda del binning ---
        if binx == 2 and biny == 2:
            roi_w, roi_h = 50, 936
            roi0_x, roi0_y = 404, 0
            dx = 60
            gap = -5
        elif binx == 1 and biny == 1:
            roi_w, roi_h = 100, 1872
            roi0_x, roi0_y = 808, 0
            dx = 120
            gap = -8
        else:
            print(f"[WARNING] Unsupported binning {binx}x{biny}. Using default parameters.")
            roi_w, roi_h = 50, 936
            roi0_x, roi0_y = 404, 0
            dx = 60
            gap = -4

        roi_origins = [
            (roi0_x, roi0_y),
            (roi0_x + dx, roi0_y),
            (roi0_x + 2 * dx + gap, roi0_y),
            (roi0_x + 3 * dx + gap, roi0_y),
        ]

        # --- 5️⃣ Esegui la pipeline ---
        process_all_files_with_interface(
            folder_path, roi_origins, roi_w, roi_h,
            dx, gap, sigma_threshold, npixels,
            max_move, snr_threshold
        )

    except Exception as e:
        print(f"Error reading FITS header or processing files: {e}")


# Note: The interactive interface will be displayed inline when selector.show() is called.
# The flow pauses at that point until the user clicks the 'CONTINUE' button.
# The processing of the next file is triggered by the button click handler.

[INFO] Found 110 FITS/FITS.GZ files in /content/drive/Shared drives/Emissary/OSSERVAZIONI_EKAR/20251028/HD42182_1st
    AF969610.fits.gz
    AF969611.fits.gz
    AF969612.fits.gz
    AF969613.fits.gz
    AF969614.fits.gz
    AF969615.fits.gz
    AF969616.fits.gz
    AF969617.fits.gz
    AF969618.fits.gz
    AF969619.fits.gz
   ... and 100 more files not shown.


Reading header from: AF969610.fits.gz
Binning detected: 1x1
Found 110 FITS/FITS.GZ files to process:
   AF969610.fits.gz
   AF969611.fits.gz
   AF969612.fits.gz
   AF969613.fits.gz
   AF969614.fits.gz
   AF969615.fits.gz
   AF969616.fits.gz
   AF969617.fits.gz
   AF969618.fits.gz
   AF969619.fits.gz
  ... and 100 more files.

[Info] Detected 5 sources in defined ROIs.
  ROI 1: x = 967.09, y = 929.02
  ROI 2: x = 859.24, y = 930.10
  ROI 3: x = 1202.12, y = 930.43
  ROI 4: x = 1094.31, y = 931.24
  ROI 5: x = 845.23, y = 1865.06


ROI 0 initial (859.2,930.1) → refined (858.8,929.9)
[DEBUG] In iterative: median=1260.00, min=1242.00, max=24603.00
ROI 1 initial (979.2,930.1) → refined (978.5,927.8)
[DEBUG] In iterative: median=1260.00, min=1242.00, max=24603.00
ROI 2 initial (1091.2,930.1) → refined (1091.4,931.4)
[DEBUG] In iterative: median=1260.00, min=1242.00, max=24603.00
ROI 3 initial (1211.2,930.1) → refined (1210.9,926.2)
[DEBUG] In iterative: median=1260.00, min=1242.00, max=24603.00
[Info] Negative flux (-330.43) at r=2.0 for (1210.9,926.2); continuing. phot_table 17800.47, bkg_mean 1442.81, aperture.area 12.57)
[INFO] Using common aperture radius = 21.00 px, annulus outer radius = 28.00 px, minimum radii was 14.00
ROI 0: flux=557076.7, Bkg=1264.43, RMS=4.37
ROI 1: flux=550928.1, Bkg=1266.21, RMS=4.94
ROI 2: flux=622267.2, Bkg=1264.79, RMS=4.06
ROI 3: flux=624818.4, Bkg=1266.85, RMS=5.40

Fluxes for AF969610.fits.gz:
   I0=557076.73, I90=550928.08, I45=622267.17, I135=624818.39
[Info] Detected 4 sources i